In [105]:
import tensorflow as tf
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings("ignore")

In [106]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image, label

scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

Buffer_Size = 4500

shuffled_train_validation_data = scaled_train_validation_data.shuffle(Buffer_Size)

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

Batch_Size = 150
train_data = train_data.batch(Batch_Size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))


2022-01-21 21:46:32.533491: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [107]:
input_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [108]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [109]:
num_epochs=25
model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/25
360/360 - 5s - loss: 0.2871 - accuracy: 0.9139 - val_loss: 0.1656 - val_accuracy: 0.9502 - 5s/epoch - 14ms/step
Epoch 2/25
360/360 - 4s - loss: 0.1349 - accuracy: 0.9595 - val_loss: 0.1375 - val_accuracy: 0.9585 - 4s/epoch - 11ms/step
Epoch 3/25
360/360 - 4s - loss: 0.0924 - accuracy: 0.9709 - val_loss: 0.0803 - val_accuracy: 0.9758 - 4s/epoch - 11ms/step
Epoch 4/25
360/360 - 4s - loss: 0.0717 - accuracy: 0.9770 - val_loss: 0.0751 - val_accuracy: 0.9778 - 4s/epoch - 12ms/step
Epoch 5/25
360/360 - 4s - loss: 0.0542 - accuracy: 0.9828 - val_loss: 0.0688 - val_accuracy: 0.9788 - 4s/epoch - 11ms/step
Epoch 6/25
360/360 - 4s - loss: 0.0431 - accuracy: 0.9863 - val_loss: 0.0636 - val_accuracy: 0.9802 - 4s/epoch - 11ms/step
Epoch 7/25
360/360 - 4s - loss: 0.0335 - accuracy: 0.9890 - val_loss: 0.0586 - val_accuracy: 0.9807 - 4s/epoch - 11ms/step
Epoch 8/25
360/360 - 4s - loss: 0.0282 - accuracy: 0.9907 - val_loss: 0.0523 - val_accuracy: 0.9842 - 4s/epoch - 11ms/step
Epoch 9/25
360/3

In [110]:
test_loss,test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

1/1 [==============================] - 0s 444ms/step - loss: 0.0989 - accuracy: 0.9787
Test loss: 0.10. Test accuracy: 97.87%
